In [1]:
!pip install -U langchain langgraph langchain_openai langchainhub tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tavily-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tavily-python to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9

In [2]:
from google.colab import userdata
import os

In [3]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

## Agent Definition

### Generating openai_functions_agent

In [5]:
tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4-1106-preview")

In [6]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [30]:
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [31]:
agent_runnable

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [32]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

In [33]:
agent

RunnableAssign(mapper={
  agent_outcome: RunnableAssign(mapper={
                   agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
                 })
                 | ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],

## Service Function

In [34]:
def execute_tools(data):
  agent_action = data.pop('agent_outcome')
  tool_to_use = {t.name: t for t in tools}[agent_action.tool]
  observation = tool_to_use.invoke(agent_action.tool_input)
  data['intermediate_steps'].append((agent_action, observation))
  return data

In [35]:
def should_continue(data):
  if isinstance(data['agent_outcome'], AgentFinish):
    return "exit"
  else:
    return "continue"

In [36]:
from langgraph.graph import END, Graph

In [37]:
workflow = Graph()

In [38]:
workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools)

In [39]:
workflow.set_entry_point("agent")

## agent - continue -> tools
             |
             -exit -> END

In [40]:
workflow.add_conditional_edges(
    "agent", #start node
    should_continue,
    {
        "continue": "tools",
        "exit": END
    }
)

In [41]:
workflow

In [42]:
workflow.add_edge('tools', 'agent')

In [43]:
chain = workflow.compile()

## Execution

In [45]:
chain.invoke({"input": "Tell me 5 startups in the field of Quantum Computing", "intermediate_steps": []})

{'input': 'Tell me 5 startups in the field of Quantum Computing',
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'top 5 startups in Quantum Computing 2023'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'top 5 startups in Quantum Computing 2023'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"top 5 startups in Quantum Computing 2023"}', 'name': 'tavily_search_results_json'}})]),
   [{'url': 'https://quantumpace.com/quantum-computing-startups/',
     'content': "Quantum Computing Startups — 2024  1QBit – quantum computing startups  Financial data – quantum computing startups  \u200bGrowth of the quantum computing startups industry\u200bQuantum Computing Startups - discover the most promising young companies in the QC industry! Qiskit. Founded in 2017 in Armonk, NY, Qiskit is an open-source platform aimed at making quantum computing more accessible. It offers